Empezamos por el principio, importamos las librerías que vamos a necesitar.

In [1]:
import os
import cv2, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions

Using TensorFlow backend.


In [1]:
import numpy as np
import keras
from keras.preprocessing import text, sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import GRU, Dropout, Dense
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [0]:
from keras.applications.inception_v3 import InceptionV3
img_model = InceptionV3(include_top=True)

Descargamos las descripciones y las fotos de nuestro dataset y las descomprimimos.

In [3]:
!wget http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_text.zip
!unzip -o Flickr8k_text.zip

--2018-08-13 19:39:35--  http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_text.zip
Resolving nlp.cs.illinois.edu (nlp.cs.illinois.edu)... 192.17.58.132
Connecting to nlp.cs.illinois.edu (nlp.cs.illinois.edu)|192.17.58.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2340801 (2.2M) [application/zip]
Saving to: ‘Flickr8k_text.zip.1’

Flickr8k_text.zip.1 100%[===================>]   2.23M  2.36MB/s    in 0.9s    

2018-08-13 19:39:36 (2.36 MB/s) - ‘Flickr8k_text.zip.1’ saved [2340801/2340801]

Archive:  Flickr8k_text.zip
  inflating: CrowdFlowerAnnotations.txt  
  inflating: ExpertAnnotations.txt   
  inflating: Flickr8k.lemma.token.txt  
  inflating: __MACOSX/._Flickr8k.lemma.token.txt  
  inflating: Flickr8k.token.txt      
  inflating: Flickr_8k.devImages.txt  
  inflating: Flickr_8k.testImages.txt  
  inflating: Flickr_8k.trainImages.txt  
  inflating: readme.txt              


In [0]:
!wget http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_Dataset.zip
!unzip -q Flickr8k_Dataset.zip

In [0]:
!ls -l

Abrimos el fichero con las descripciones y las volcamos en un diccionario tratándolas previamente.

In [0]:
fname = 'Flickr8k.token.txt'
img_to_caps = dict()

with open(fname, 'r') as f:
    for line in f:
        tokens = line.split(' ')
        img_fname, num = tokens[0].split('#')
        caption = ' '.join(tokens[1:]).strip()
        if img_fname not in img_to_caps: img_to_caps[img_fname] = []
        img_to_caps[img_fname].append(caption)

In [0]:
n_vocab = 100

In [0]:
tk = text.Tokenizer(nb_words=n_vocab)

texts = []
for img_name in img_to_caps:
    texts += img_to_caps[img_name]

tk.fit_on_texts(texts)
sorted_word_counts = sorted(tk.word_counts.items(), key=lambda x: x[1])
sorted_word_counts = sorted_word_counts[::-1][:n_vocab]

In [0]:
sorted_word_counts

Tokenizamos y ordenamos las palabras.

In [15]:
tk = text.Tokenizer(nb_words=n_vocab)

texts = []
for img_name in img_to_caps:
    texts += img_to_caps[img_name]

tk.fit_on_texts(texts)
sorted_word_counts = sorted(tk.word_counts.items(), key=lambda x: x[1])
sorted_word_counts = sorted_word_counts[::-1][:n_vocab]

/usr/local/lib/python3.6/dist-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [0]:
word_to_int = {t[0]: i for i,t in enumerate(sorted_word_counts)}
int_to_word = {i: t[0] for i,t in enumerate(sorted_word_counts)}

In [0]:
max_seq_len = 16

Creamos las secuencias.

In [0]:
img_to_seqs = {}
for img_fname, captions in img_to_caps.items():
    seqs = []
    for caption in captions:
        seqs.append([word_to_int[w] for w in caption.split() if w in word_to_int])
    img_to_seqs[img_fname] = seqs

In [0]:
img_to_padded_seqs, img_to_next_chars = {}, {}
for img_fname, seqs in img_to_seqs.items():
    partial_seqs = []
    next_words = []
    for seq in seqs:
        for i in range(1,len(seq)):
            partial_seqs.append(seq[:i])
            next_words.append(seq[i])
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_seq_len)
    
    next_words_1hot = np.zeros([len(next_words), n_vocab], dtype=np.bool)
    for i,next_word in enumerate(next_words):
        next_words_1hot[i,next_word] = 1
    
    img_to_padded_seqs[img_fname] = padded_partial_seqs
    img_to_next_chars[img_fname] = next_words_1hot

Empezamos con nuestro modelo. Como RNN vamos a usar GRU.

In [0]:
X_lst, y_lst = [], []
for img_fname in img_to_padded_seqs:
    X_lst.append(img_to_padded_seqs[img_fname])
    y_lst.append(img_to_next_chars[img_fname])

In [0]:
X, y = np.concatenate(X_lst, axis=0), np.concatenate(y_lst, axis=0)
X = np.expand_dims(X, axis=2)

In [0]:
model = Sequential()
model.add(GRU(256, input_shape=(max_seq_len,1)))
model.add(Dense(n_vocab,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
model_checkpoint = ModelCheckpoint('weights.{epoch:02d}-{loss:.2f}.hdf5', monitor='loss', verbose=0,
                                  save_best_only=True, mode='min')
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False)
callbacks_list = [model_checkpoint, tensorboard]

Entrenamos nuestro modelo.

In [34]:
nb_epoch = 20
batch_size = 128


model.fit(X, y, nb_epoch=nb_epoch, batch_size=batch_size)

Epoch 1/20
   128/219751 [..............................] - ETA: 3:06 - loss: 2.9453

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


219751/219751 [==============================] - 165s 749us/step - loss: 2.8887
Epoch 2/20
219751/219751 [==============================] - 163s 743us/step - loss: 2.8447
Epoch 3/20
219751/219751 [==============================] - 158s 719us/step - loss: 2.8091
Epoch 4/20
219751/219751 [==============================] - 160s 728us/step - loss: 2.7774
Epoch 5/20
219751/219751 [==============================] - 160s 727us/step - loss: 2.7477
Epoch 6/20
219751/219751 [==============================] - 157s 713us/step - loss: 2.7203
Epoch 7/20
219751/219751 [==============================] - 157s 715us/step - loss: 2.6947
Epoch 8/20
219751/219751 [==============================] - 157s 717us/step - loss: 2.6718
Epoch 9/20
219751/219751 [==============================] - 159s 722us/step - loss: 2.6493
Epoch 10/20
219751/219751 [==============================] - 159s 724us/step - loss: 2.6280
Epoch 11/20
219751/219751 [==============================] - 160s 728us/step - loss: 2.6102
Epoch 12

In [1]:
!ls -l

total 1101892
-rw-r--r-- 1 root root    2918552 Oct 14  2013 CrowdFlowerAnnotations.txt
drwxr-xr-x 3 root root       4096 Aug 13 21:16 datalab
-rw-r--r-- 1 root root     346674 Oct 14  2013 ExpertAnnotations.txt
drwxr-xr-x 2 root root     417792 Oct  3  2012 Flicker8k_Dataset
-rw-r--r-- 1 root root 1115419746 Oct 24  2013 Flickr8k_Dataset.zip
-rw-r--r-- 1 root root      25801 Oct 10  2013 Flickr_8k.devImages.txt
-rw-r--r-- 1 root root    3244761 Feb 16  2012 Flickr8k.lemma.token.txt
-rw-r--r-- 1 root root      25775 Oct 10  2013 Flickr_8k.testImages.txt
-rw-r--r-- 1 root root    2340801 Oct 28  2013 Flickr8k_text.zip
-rw-r--r-- 1 root root    3395237 Oct 14  2013 Flickr8k.token.txt
-rw-r--r-- 1 root root     154678 Oct 10  2013 Flickr_8k.trainImages.txt
drwxrwxr-x 3 root root       4096 Aug 14 18:47 __MACOSX
-rw-r--r-- 1 root root       1821 Oct 14  2013 readme.txt


Ahora vamos a cargar en nuestro modelos los pesos generados en un paso anterior. Para probarlo intentamos generar una salida aleatoria.

In [0]:
# Loading model from weight

weight_fname = 'weights.01-2.95.hdf5'
model.load_weights(weight_fname)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:
# Generate random start

num_iter = 100
curr_seq = np.zeros([1,max_seq_len,1])
curr_seq[0,:,0] = X[np.random.randint(X.shape[0])].reshape(1,-1)

for i in curr_seq[0,:,0]:
    if i != 0: print(sorted_word_counts[int(i)][0], end=' ')
print()

for i in range(num_iter):
    prediction = model.predict(curr_seq)
    idx = np.argmax(prediction)
    next_word = sorted_word_counts[idx][0]
    curr_seq[0,:max_seq_len-1,0] = curr_seq[0,1:,0]
    curr_seq[0,max_seq_len-1,0] = idx
    print(next_word, end=' ')


dog through 
through a field with a in its mouth mouth the water the beach in the background her her her her a man in a blue shirt is a ball in front of a white people and blue on the grass a man water and blue shirt on a beach in front of a white people and a blue shirt in the background her a man in a blue shirt and a blue shirt in the background her a ball in front of a white and a red shirt and a blue shirt on a beach with people in the background 